In [ ]:
#scenarios

# find earliest and latest dates in the month
# find earliest date in the month, not latest date.  Set latest date to 31
# find latest date in the month, not earliest.  This is the same as the above
# 
# what if 2021-07-10 and 2021-08-06?  If month not equal, earliest = 01 and latest = 06 

In [1]:
date1 = "2021-08-01"
date2 = "2021-08-05"


wipByDay = {"1":0, "2":0, "3":0, "4":0, "5":0}

In [2]:
earliest = date1[-2:]
latest = date2[-2:]
print(earliest)
print(latest)
int_earliest = int(earliest)
print(int_earliest)
int_latest = int(latest)
print(int_latest)
print(range(int_earliest, int_latest))

month1 = date1[-5:-3]

print(month1)

01
05
1
5
range(1, 5)
08


In [26]:
def updateWipByDay(earliest_date, latest_date):

    for n in range(int(earliest_date), int(latest_date)): #don't count last date

        wipByDay[str(n)] +=1 
        #print(f."wip day {n} is {wipByDay[str(n)]}")

        #print("wip day " + str(n) + " is " + str(wipByDay[str(n)]) )

In [158]:
updateWipByDay("03","02")

In [153]:
updateWipByDay("02","03")

wip day 2 is 1
wip day 3 is 1


In [96]:
from datetime import date




def calculateFlowTime(startDate, endDate):
    
    fdate = date(int(startDate[0:4]),int(startDate[5:7]),int(startDate[-2:]))
    
    print(fdate)
    ldate = date(int(endDate[0:4]),int(endDate[5:7]),int(endDate[-2:]))
    print(ldate)
    delta = ldate - fdate
    print(delta.days)
    
    return(delta.days)

dateTest = "2021-03-20"

doneDateTest = "2021-05-10"

calculateFlowTime(dateTest, doneDateTest)

2021-03-20
2021-05-10
51


In [154]:
print(wipByDay)

{'1': 0, '2': 1, '3': 1, '4': 0, '5': 0}


In [137]:
# working cell

import csv
from datetime import date
import statistics

def convertDateFomat(jiraDate):
    year = int(jiraDate[0:4])
    month = int(jiraDate[5:7])
    day = int(jiraDate[-2:])
    
    formattedDate = date(year, month, day)
    
    return(formattedDate)

def calculateFlowTime(startDate, endDate):
    
    fdate = date(int(startDate[0:4]),int(startDate[5:7]),int(startDate[-2:]))
    
    ldate = date(int(endDate[0:4]),int(endDate[5:7]),int(endDate[-2:]))

    delta = ldate - fdate

    print(f"start date {fdate} done date{ldate} flow time {delta.days}")
    
    return(delta.days)

def updateWipByDay(earliest_date, latest_date):

    for n in range(int(earliest_date), int(latest_date)+1): #don't count last date

        wipByDay[str(n)] +=1 
        #print(f."wip day {n} is {wipByDay[str(n)]}")
        #print("wip day " + str(n) + " is " + str(wipByDay[str(n)]) )

def findMaxWip(wipDictionary):
    
    maxWip = 0
    maxWipDay = ""
    
    for n in wipDictionary:
    
        if wipDictionary[n] > maxWip:
        
            maxWip = wipDictionary[n]
        
            maxWipDay = n
    
    maxWipDayAndCount = {}
    
    maxWipDayAndCount[str(maxWipDay)] = maxWip
    
    return maxWipDayAndCount

def city_country(city, country):
   location = {}
   location[city] = country

defaultCurrentMonth = "08"

print(f"Current default month {defaultCurrentMonth}")

currentMonth = input("Enter a month in two digit form, e.g. 08: ")

if currentMonth == "" : currentMonth = defaultCurrentMonth

flowTimes = []

fileName = "/Users/richard.holloway/Development/hmrc/jira-to-analytics/dlk-output.csv"

inProgressColumn = 5
deployingColumn = 6
deployedColumn = 7
doneColumn = 8

ignoreList = ["DLK-853", "DLK-854"]

wipByDay = {}

for n in range(1,32):
    
    wipByDay[str(n)] = 0

with open(fileName, newline='') as csvfile:
    
    numberOfRows = 0
        
    for line in csvfile.readlines():
        array = line.split(',')
        numberOfRows +=1 
        
        jiraId = array[0]
        
        dateRangeForRow = []

        # some things to skip
        
        if array[0] == "ID": # skip header row
            continue
            
        if array[0] in ignoreList:
            print(f"{array[0]} in ignoreList")
            continue
            
        if array[inProgressColumn] == "" and array[deployingColumn] == "" and array[deployedColumn] == "" and array[doneColumn] =="": 
            print(f"{array[0]} blank row skipped")
            continue
            
        # skip if done ticket in previous month, this can happen if someone has updated commented on a done ticket
        
        if array[doneColumn] != "" and array[doneColumn][-5:-3] != currentMonth:
            print(f"{array[0]} ticket closed in previous month, row skipped")
            continue

        if array[inProgressColumn] != "":
            dateRangeForRow.append(array[inProgressColumn])
            
        if array[deployingColumn] != "":
            dateRangeForRow.append(array[deployingColumn])
            
        if array[deployedColumn] != "":
            dateRangeForRow.append(array[deployedColumn])
            
        if array[doneColumn] != "":
            dateRangeForRow.append(array[doneColumn])
            
        #print(dateRangeForRow)
        
        earliestDate = min(dateRangeForRow)
        latestDate = max(dateRangeForRow)
        
        convertedEarliestDate = convertDateFomat(earliestDate)
        convertedLatestDate = convertDateFomat(latestDate)
        
        delta = convertedLatestDate - convertedEarliestDate
        
        if array[doneColumn] != "":
            
            flowTime = delta.days
            
            flowTimes.append(flowTime)
        
            print(f"{jiraId} earliest date {earliestDate}, latest date {latestDate}, flowTime = {flowTime}")
        
        else:
            
            print(f"{jiraId} earliest date {earliestDate}, latest date {latestDate}")
        
        if convertedEarliestDate.month != int(currentMonth):
            
            wipEarliestDate = "01"
        
        else:
            
            wipEarliestDate = str(convertedEarliestDate.day)
            
        if array[doneColumn] == "":
            
            latestWipDate = "31"
        
        else:
            
            latestWipDate = str(convertedLatestDate.day)
            
        #print(f"e wip day {wipEarliestDate} l wip day {latestWipDate}")
        
        updateWipByDay(wipEarliestDate, latestWipDate)
            
print()

wip = findMaxWip(wipByDay)

for key in wip:
    print(f"max wip {wip[key]} on day {key}")

            
print()
print(f"average flow time = {statistics.mean(flowTimes)}")        
print(f"stdev flow time = {statistics.stdev(flowTimes)}")


        


Current default month 08
Enter a month in two digit form, e.g. 08: 
DLK-552 earliest date 2021-03-12, latest date 2021-03-12
DLK-639 blank row skipped
DLK-654 ticket closed in previous month, row skipped
DLK-723 earliest date 2021-06-01, latest date 2021-06-01
DLK-791 earliest date 2021-06-14, latest date 2021-06-14
DLK-795 earliest date 2021-06-08, latest date 2021-08-17, flowTime = 70
DLK-803 blank row skipped
DLK-815 earliest date 2021-06-11, latest date 2021-08-17, flowTime = 67
DLK-818 earliest date 2021-06-29, latest date 2021-06-29
DLK-827 earliest date 2021-08-17, latest date 2021-08-25, flowTime = 8
DLK-828 earliest date 2021-07-22, latest date 2021-08-17, flowTime = 26
DLK-832 earliest date 2021-07-21, latest date 2021-08-17, flowTime = 27
DLK-833 earliest date 2021-08-17, latest date 2021-08-18
DLK-839 earliest date 2021-07-21, latest date 2021-08-02, flowTime = 12
DLK-845 earliest date 2021-09-01, latest date 2021-09-01
DLK-848 earliest date 2021-07-30, latest date 2021-08-